In [6]:
# define a model using pytorch
# a pretrained model adding layers to classify the 4 classes of the images
import torch
import torch.nn as nn
from torchvision import models

class MIDmodel(nn.Module):
    def __init__(self):
        super(MIDmodel, self).__init__()
        # first pretrained
        self.pretrained = models.resnet18(pretrained=True)
        # add layers
        self.fc1 = nn.Linear(1000, 128)
        self.fc2 = nn.Linear(128, 4)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # set x to three channels
        x = x.repeat(1, 3, 1, 1)
        x = self.pretrained(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [7]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
import os
class ImageDataset(Dataset):
    def __init__(self, root_dir, patients_ids, patients_df, transform=None):
        # select the rows of the dataframe that correspond to the patients in the list
        self.patients_df = patients_df[patients_df["patient_id"].isin(patients_ids)]
        self.root_dir = root_dir

    def __len__(self):
        return len(self.patients_df)

    def __getitem__(self, idx):
        # get idx row of the dataframe
        row = self.patients_df.iloc[idx]
        # path = root_dir/patientid_examid_spot_framenumber_score.png
        path = os.path.join(self.root_dir, row["patient_id"] + "_" + row["exam_id"] + "_" + row["spot"] + "_" + row["frame_number"] + "_" + row["score"] + ".png")
        image = Image.open(path)
        # Extract label from the image filename
        label = row["score"]
        # set the image to a tensor
        image = transforms.ToTensor()(image)
        return image, int(label)

In [8]:
# define dataset
import pandas as pd
from pandas_profiling import ProfileReport
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# "patient_id exam_id spot[1-14] frame_number score"
# load patients data in a dataframe from images folder
import glob
images_paths = glob.glob("images/*.png", recursive=True)
# images are named as: patientid_examid_spotnumber_framenumber_score.png
# create a dataframe with the data removing "images/"
images_df = pd.DataFrame([path[7:-4].split("_") for path in images_paths], columns=["patient_id", "exam_id", "spot", "frame_number", "score"])
images_df["score"] = images_df["score"].astype(str)
images_df["frame_number"] = images_df["frame_number"].astype(str)
images_df["spot"] = images_df["spot"].astype(str)
images_df["patient_id"] = images_df["patient_id"].astype(str)
images_df["exam_id"] = images_df["exam_id"].astype(str)

# profile = ProfileReport(images_df, title='Pandas Profiling Report', html={'style':{'full_width':True}})
# profile.to_file("images_df.html")

# print for each patient the number of images
# print(images_df.groupby("patient_id").count()["score"])

patients_ids = set(images_df["patient_id"])
patients_ids = list(patients_ids)
# sort the list of patients
patients_ids.sort()
# import random
# random.shuffle(patients_ids)
# train_patients = patients_ids[:int(len(patients_ids)*0.8)]
# test_patients = patients_ids[int(len(patients_ids)*0.8):]
# print("train patients: ", train_patients, "test patients: ", test_patients)
# train_dataset = ImageDataset("images/", train_patients, images_df)
# test_dataset = ImageDataset("images/", test_patients, images_df)
# print("train dataset length: ", len(train_dataset), "test dataset length: ", len(test_dataset))

# patients ['1068', '1017', '1045', '1066', '1067', '1052', '1050', '1069', '1047', '1048', '1051']
# all possible configurations with 8 patients for training and 3 for testing (56)
folds_test = []
folds_train = []
for i in range(0, 8):
    for j in range(i+1, 9):
        for k in range(j+1, 10):
            folds_test.append([patients_ids[i], patients_ids[j], patients_ids[k]])
            # add the other patients to the training set
            folds_train.append([patients_ids[x] for x in range(0, 10) if x not in [i, j, k]])
# print("folds test: ", folds_test, "folds train: ", folds_train)
# train_dataset = ImageDataset("images/", folds_train[0], images_df)
# test_dataset = ImageDataset("images/", folds_test[0], images_df)


In [9]:
import numpy as np
from torch import optim
from tqdm import tqdm

def train_model(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects/ len(dataloader.dataset)
    return epoch_loss, epoch_acc

def test_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects / len(dataloader.dataset)
    return epoch_loss, epoch_acc


In [10]:
data_dir = "images"
num_epochs = 3
learning_rate = 0.001

# define the model


for i in range(0, len(folds_test)):
    model = MIDmodel()
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    print("Fold: ", i)
    print("Test patients: ", folds_test[i])
    train_dataset = ImageDataset("images/", folds_train[i], images_df)
    test_dataset = ImageDataset("images/", folds_test[i], images_df)
    # define the dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    results_train = []
    results_test = []
    # train the model
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}")
        train_loss, train_acc = train_model(model, train_dataloader, optimizer, criterion, device)
        results_train.append([train_loss, train_acc])
        print(f"Training Loss: {train_loss:.4f} Training Accuracy: {train_acc:.4f}")
        test_loss, test_acc = test_model(model, test_dataloader, criterion, device)
        results_test.append([test_loss, test_acc])
        print(f"Test Loss: {test_loss:.4f} Test Accuracy: {test_acc:.4f}")
    print(f"Results fold {i}:")
    # print results with commas not dots
    print("Train: ", results_train)
    print("Test: ", results_test)

    # save the model
    torch.save(model.state_dict(), f"model{i}.pt")



c:\Users\Davide\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Davide\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold:  0
Test patients:  ['1017', '1045', '1047']
Epoch 0


100%|██████████| 1965/1965 [07:05<00:00,  4.62it/s]


Training Loss: 0.2941 Training Accuracy: 0.8917


100%|██████████| 807/807 [01:16<00:00, 10.61it/s]


...

Training Loss: 0.0835 Training Accuracy: 0.9709


100%|██████████| 807/807 [01:23<00:00,  9.61it/s]


Test Loss: 3.6084 Test Accuracy: 0.3128
Results fold 0:
Train:  [[0.2940609783029931, tensor(0.8917, device='cuda:0')], [0.12015317365912966, tensor(0.9581, device='cuda:0')], [0.08352938765807037, tensor(0.9709, device='cuda:0')]]
Test:  [[3.096125132599891, tensor(0.3852, device='cuda:0')], [2.2399193264143666, tensor(0.4455, device='cuda:0')], [3.608429243206525, tensor(0.3128, device='cuda:0')]]
Fold:  1
Test patients:  ['1017', '1045', '1048']
Epoch 0


 12%|█▏        | 197/1637 [00:46<05:39,  4.24it/s]


KeyboardInterrupt: 

In [ ]:
# load the model
model = MIDmodel()
model.load_state_dict(torch.load("model.pt"))
model.eval()